In [ ]:
!pip install -U pandasql

# Data Analysis for GOS Case #1 with Georg Fischer AG

In [ ]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from pandasql import sqldf
import itertools
from itertools import product
import requests
import json
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df_complete_dataset_cost_breakdown = pd.read_csv('cost_breakdown.csv', 
                                                 delimiter=';', 
                                                 encoding= 'unicode_escape', 
                                                 on_bad_lines='skip')
df_complete_dataset_market_view = pd.read_csv('market_view.csv', 
                                              delimiter=';', 
                                              encoding= 'unicode_escape', 
                                              on_bad_lines='skip')

## 1) Sales Prediction

In [ ]:
## Prediction V3:

# Sum Sales 2022 and put into a column:
Quantity_Columns_2022 = ['Invoiced Quant. (BU)', 'Invoiced Quant. (BU).1', 'Invoiced Quant. (BU).2',
                      'Invoiced Quant. (BU).3', 'Invoiced Quant. (BU).4', 'Invoiced Quant. (BU).5',
                      'Invoiced Quant. (BU).6', 'Invoiced Quant. (BU).7', 'Invoiced Quant. (BU).8',
                      'Invoiced Quant. (BU).9', 'Invoiced Quant. (BU).10', 'Invoiced Quant. (BU).11']

Net_Value_Columns_2022 = ['Net Value (GC)', 'Net Value (GC).1', 'Net Value (GC).2',
                          'Net Value (GC).3', 'Net Value (GC).4', 'Net Value (GC).5',
                          'Net Value (GC).6', 'Net Value (GC).7', 'Net Value (GC).8',
                          'Net Value (GC).9', 'Net Value (GC).10', 'Net Value (GC).11']
# print(df_complete_dataset_market_view[Net_Value_Columns_2022])

# Replace 'PC' character with empty string for Unites
df_complete_dataset_market_view[Quantity_Columns_2022] = (
    df_complete_dataset_market_view[Net_Value_Columns_2022].replace("PC", "", regex=True))

# Replace ' character with empty string for Sales
df_complete_dataset_market_view[Net_Value_Columns_2022] = (
    df_complete_dataset_market_view[Net_Value_Columns_2022].replace("'", "", regex=True))

# Convert sales columns to numeric values
df_complete_dataset_market_view[Net_Value_Columns_2022] = (
    df_complete_dataset_market_view[Net_Value_Columns_2022].apply(pd.to_numeric, errors='coerce'))

# Summing up the monthly sales to total yearly sales
df_complete_dataset_market_view['Total_Sales_2022'] = (
    df_complete_dataset_market_view[Net_Value_Columns_2022].sum(axis=1, skipna=True))
df_complete_dataset_market_view['Total_Unites_2022'] = (
    df_complete_dataset_market_view[Quantity_Columns_2022].sum(axis=1, skipna=True))

# Add columns for predicted sales from 2022 to 2031
for year in range(2022, 2032):
    col_name = f'Predicted Sales {year}'
    growth_rate = df_complete_dataset_market_view['Growth rate untill 2031']  # f'ing typo
    total_sales = df_complete_dataset_market_view['Total_Sales_2022']
    growth_rate_numeric = (abs(pd.to_numeric(growth_rate.str.strip('%'))))**(0.1) / 100
    predicted_sales = total_sales * (1 + growth_rate_numeric) ** (year - 2022)
    df_complete_dataset_market_view[col_name] = predicted_sales

df_complete_dataset_market_view.head(10)


## 2) CoG Light

In [ ]:
# Input:  Dataframe containing the destinations and the weighting, everything already needs to be added up per 
#         country, function does not necessarily need to contain the latitude and longitude since I will take this 
#         from the relevant countries list
# Output: Weighted gravity

def CoG_light_overhead_sales(df_all_deliveries, str_name_col_dest, str_name_col_weight):
    
    weighted_sum_lat = 0.0
    weighted_sum_lon = 0.0
    sum_weight_total = 0.0
    
    ## Analysis for total sales
    #for row in df_all_deliveries.iterrows():
    #     if row[1][str_name_col_dest] in rel_country_lst:
    #        sum_weight_total += row[1][str_name_col_weight]
    #        weighted_sum_lon += relevant_countries.loc[relevant_countries['Country'] == (
    # row[1][str_name_col_dest],'Longitude'].tolist()[0]*(row[1][str_name_col_weight]))
    #        weighted_sum_lat += relevant_countries.loc[relevant_countries['Country'] == (
    # row[1][str_name_col_dest],'Latitude'].tolist()[0]*(row[1][str_name_col_weight]))
    
    ## Analysis for just the overhead
    for row in df_all_deliveries.iterrows():
            overhead = row[1][str_name_col_weight]-row[1]['Total_Sales_2022']
            sum_weight_total += overhead
            weighted_sum_lon += relevant_countries.loc[relevant_countries['Country'] == (
                row[1][str_name_col_dest],'Longitude'].tolist()[0]*overhead)
            weighted_sum_lat += relevant_countries.loc[relevant_countries['Country'] == (
                row[1][str_name_col_dest],'Latitude'].tolist()[0]*overhead)
    
    return weighted_sum_lat/sum_weight_total, weighted_sum_lon/sum_weight_total

In [ ]:
# Input:  Dataframe containing the destinations and the weighting, everything already needs to be added up per 
#         country, function does not necessarily need to contain the latitude and longitude since I will take this 
#         from the relevant countries list
# Output: Weighted gravity

def CoG_light_total_sales(df_all_deliveries, str_name_col_dest, str_name_col_weight):
    
    weighted_sum_lat = 0.0
    weighted_sum_lon = 0.0
    sum_weight_total = 0.0
    
    ## Analysis for total sales
    for row in df_all_deliveries.iterrows():
            sum_weight_total += row[1][str_name_col_weight]
            weighted_sum_lon += (
            relevant_countries.loc[relevant_countries['Country'] == row[1][str_name_col_dest], 
                                   'Longitude'].tolist()[0]
                                    * (row[1][str_name_col_weight]))

            weighted_sum_lat += (relevant_countries.loc[relevant_countries['Country'] == row[1][str_name_col_dest],
                                    'Latitude'].tolist()[0]
                                    * (row[1][str_name_col_weight]))


    return weighted_sum_lat/sum_weight_total, weighted_sum_lon/sum_weight_total

In [ ]:
relevant_countries = pd.read_csv('relevant_countries.txt', 
                                 delimiter=',', 
                                 encoding= 'unicode_escape', 
                                 on_bad_lines='skip')

In [ ]:
## Make a dataframe that only includes the relevant destinations

relevant_orders = df_complete_dataset_market_view.loc[
    df_complete_dataset_market_view['Country Ship-To'].isin(
        relevant_countries['Country'])]

In [ ]:
# Define an array of all the data you want to look at

weighting_parameter = [
    'Predicted Sales 2023',
    'Predicted Sales 2024',
    'Predicted Sales 2025',
    'Predicted Sales 2026',
    'Predicted Sales 2027',
    'Predicted Sales 2028',
    'Predicted Sales 2029',
    'Predicted Sales 2030',
    'Predicted Sales 2031',
]

In [ ]:
optimal_coordinates = np.zeros((len(weighting_parameter), 2))

In [ ]:
for i, yearly_weighting_param in enumerate(weighting_parameter):
    lat_i, lon_i = CoG_light_total_sales(relevant_orders, "Country Ship-To", yearly_weighting_param)
    optimal_coordinates[i][0] = lat_i
    optimal_coordinates[i][1] = lon_i
    print(lat_i, lon_i)

In [ ]:
CoG_per_year = pd.DataFrame()
CoG_per_year['year'] = weighting_parameter
CoG_per_year['latitude'] = optimal_coordinates.T[0]
CoG_per_year['longitude'] = optimal_coordinates.T[1]
CoG_per_year

In [ ]:
CoG_per_year.to_csv('Simple_CoG.csv')

In [ ]:
fig = px.scatter_mapbox(
    CoG_per_year.tail(1),
    lat="latitude",
    lon="longitude",
    hover_name="year",
    color='year',
    size=[10],
    zoom=3,
    height=300
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## 3) Fancy CoG Analysis

### 3.1) Choose City Combinations
- Arbitrarily chose a number of n cities that fullfill the following requrements
    - located in eastern europe
    - population size > 5000
- Make the combinations with the destination countries in eastern europe

### 3.2) Calculate distance
- Using the OSRM API, calculate the distance between all these combinations

### 3.3) Find the weighing factor for each destination location
- Assign a volume coefficient and a material coefficient
    - Volume coefficient: in range[1, ..., 3] depending on the volume of the good
    - Material coefficient: in range[1, ..., 3] depending on the weight of the good
- Itherate through the market_view dataset and calculate the weighing factor for each order
- Weighing factor:

        $Factor = #Pieces*Material_Coefficient*Volume_Coefficient$

- We calculate this for the overhead of the production compared to current production rate for each year.
- As we will need the weighing factor per destination, we then summarize the weighing factor for all orders that go to a certain destination.

### 3.4) Make a minimization analysis
- Here we simply multiply the travelling distance calculated in step 2 with the weighing factor.
- The best result will then be the one with the heat map

### 3.5) Map the changes
- These results would look absolutely stunning on a heat map
- We can also map everything from best to worst places and how the projections would look like over the years
- https://predictivehacks.com/how-to-create-heatmap-on-a-map-in-python/

## 3.1) Choose city combinations

In [ ]:
all_european_cities = pd.read_csv(
    'geonames-all-european-cities.csv',
    delimiter=';',
    encoding='unicode_escape',
    on_bad_lines='skip'
)

In [ ]:
# City should have > 5k inhabitants and be contained in the relevant countries list
reduced_city_list = all_european_cities[
    (all_european_cities['Population'] > 5000) &
    (all_european_cities['Country name'].isin(relevant_countries['Country']))
]

## 3.2) Calculate the distance

## !!! Attention !!! 
Reduced_city_list still needs to be samples. Otherwise the model explodes. Sampling 20/3721 cities in eastern europe leads to 420 combinations which takes the distance calculation API ~3min to generate.

In [ ]:
selected_cities = pd.DataFrame()
num_cities_per_country = 20
for row in relevant_countries['Country']:
    temp_df = reduced_city_list[reduced_city_list['Country name'].isin([row])]
    
    if(len(temp_df)>num_cities_per_country):
        selected_cities = pd.concat([selected_cities, temp_df.sample(num_cities_per_country)])
    else:
        selected_cities = pd.concat([selected_cities, temp_df])
    
print(len(selected_cities))
print(selected_cities.head(3))

In [ ]:
# Get all unique permutations for driving distances
# Todo: Do we really want to consider countries within the same region?

list1 = relevant_countries['Capital'].tolist()
list2 = selected_cities['Name'].tolist()

city_pairs = list(itertools.product(list1, list2))
print(len(city_pairs))

In [ ]:
fig = px.scatter_mapbox(selected_cities, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Name", 
                        color='Country name', 
                        zoom=3, 
                        height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df_city_combinations = pd.DataFrame()
df_city_combinations["Potential Factory Placement"] = np.array(city_pairs).T[1]
df_city_combinations["Destination"] = np.array(city_pairs).T[0]
df_city_combinations

In [ ]:
# Find the distance between all these combinations and append to the dataframe

dist_lst = []
i = 0

for row in df_city_combinations.iterrows():
    lon_1 = relevant_countries.loc[relevant_countries['Capital'] == row[1]['Destination'],'Longitude'].tolist()[0]
    lat_1 = relevant_countries.loc[relevant_countries['Capital'] == row[1]['Destination'],'Latitude'].tolist()[0]
    lon_2 = (
    selected_cities.loc[
        selected_cities['Name'] == row[1]['Potential Factory Placement'],
        'Longitude'].tolist()[0])
    lat_2 = (
    selected_cities.loc[
        selected_cities['Name'] == row[1]['Potential Factory Placement'],
        'Latitude'].tolist()[0])
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")
    routes = json.loads(r.content)
    best_route = routes.get("routes")[0]
    dist_lst.append(best_route["distance"])
    i += 1
    print(i, " of ", len(df_city_combinations))
    
df_city_combinations['Distance'] = dist_lst

In [ ]:
df_city_combinations['Distance'] = dist_lst
avg_dist = np.average(df_city_combinations['Distance'])
df_city_combinations['Distance_normalized'] = [i/avg_dist for i in df_city_combinations['Distance']]
df_city_combinations.sample(10)

In [ ]:
df_city_combinations.to_csv('df_city_combinations'+str(len(df_city_combinations))+'.csv')

## 3.3) Find the weighing factor for each destination location

In [ ]:
weight_classification = pd.read_csv('weight_classification.CSV', 
                                    delimiter=';', 
                                    encoding= 'unicode_escape', 
                                    on_bad_lines='skip')
volume_classification = pd.read_csv('volume_classification.CSV', 
                                    delimiter=';', 
                                    encoding= 'unicode_escape', 
                                    on_bad_lines='skip')

In [ ]:
weight_classification['density_normalized'] = [
    i / np.average(weight_classification['density'])
    for i in weight_classification['density']
]

volume_classification['volume_normalized'] = [
    i / np.average(volume_classification['vol_per_item'])
    for i in volume_classification['vol_per_item']
]

In [ ]:
weight_classification.head(2)

In [ ]:
volume_classification.head(2)

In [ ]:
### Assigne the volume and the weights to the general table for all offers

lst_vol_classifcation_per_order = []
lst_weight_classifcation_per_order = []

for row in relevant_orders.iterrows():
    try: 
        v = (
    volume_classification.loc[
        volume_classification['product_group'] == str(row[1]['Product Family Short']),
        'volume_normalized'
    ].to_list()[0]
)

    except:
        v = 1.0
    
    try:
        w = (
    weight_classification.loc[
        weight_classification['Material'] == row[1]['Material'],
        'density_normalized'
    ].to_list()[0]
)

    except:
        w = 1.0
    
    lst_vol_classifcation_per_order.append(v)
    lst_weight_classifcation_per_order.append(w)
        
relevant_orders['weight_classification'] = lst_weight_classifcation_per_order
relevant_orders['volume_classification'] = lst_vol_classifcation_per_order

In [ ]:
weight_volume = 3.0
weight_weight = 1.0

weight_vol_norm = weight_volume/(weight_volume+weight_weight)
weight_wei_norm = weight_weight/(weight_volume+weight_weight)

def valuation_factor(sales_avg_comp, vol_avg_comp, weight_avg_comp):
    return sales_avg_comp*(weight_vol_norm*weight_avg_comp + weight_wei_norm*vol_avg_comp)

In [ ]:
lst_sales_overhead = []
for row in relevant_orders.iterrows():
    sales_overhead = row[1]['Predicted Sales 2031']-row[1]['Total_Sales_2022']
    lst_sales_overhead.append(sales_overhead)
    
relevant_orders['sales_overhead_2031'] = lst_sales_overhead

avg_sales_overhead = np.mean(lst_sales_overhead)

lst_sales_overhead_normalized = [i/avg_sales_overhead for i in lst_sales_overhead]
relevant_orders['sales_overhead_2031_normalized'] = lst_sales_overhead_normalized

In [ ]:
final_weighing_factor_lst = []
for row in relevant_orders.iterrows():
    _w = valuation_factor(row[1]['sales_overhead_2031_normalized'], 
                          row[1]['volume_classification'], 
                          row[1]['weight_classification'])
    final_weighing_factor_lst.append(_w)
    
relevant_orders['final_weiging_factor'] = final_weighing_factor_lst

df_score = reduced_city_list.loc[reduced_city_list['Name'].isin(df_city_combinations['Potential Factory Placement'])].drop_duplicates()
df_score

In [ ]:
### Now do the weighting
### We need a dataframe with all the potential cities, their coordinates
df_score = selected_cities
score_lst = []

i = 0

for city_samples in df_score.iterrows():
    i += 1
    potential_location_score = 0.0
    # take distance normalized from df_city_combinations at dest
    
    # for all potential cityies we are looking at
    # for all orders
        # take the distance inbetween the city that we are looking at and the destination of the order
    
    for orders in relevant_orders.iterrows():
        capital_city = (
    relevant_countries.loc[relevant_countries['Country'] == orders[1]['Country Ship-To'],'Capital'].values[0])

        potential_location_score += (df_city_combinations.loc[
        df_city_combinations['Destination'] == capital_city
    ].loc[df_city_combinations['Potential Factory Placement'] == city_samples[1]['Name'],'Distance_normalized']
    .values[0]* orders[1]['final_weiging_factor'])

    
    print(i, " of ", len(city_samples))
    
    score_lst.append(potential_location_score)
                    
df_score['score'] = score_lst
df_score['score_inverted'] = [1/i for i in score_lst]

In [ ]:
df_score.head(3)

In [ ]:
print(df_score.loc[df_score['score'] == min(df_score['score'])])

In [ ]:
print(max(df_score['score']))
print(min(df_score['score']))

In [ ]:
final_df = df_score.sort_values(by=['score'], ascending=True).head(10)

In [ ]:
final_df.to_csv('score.csv')

In [ ]:
avg = np.mean(df_score['score_inverted'])
df_score['score_inverted_normalized'] = [i/avg for i in df_score['score_inverted']]

In [ ]:
df_score.head(2)

In [ ]:
fig = px.scatter_mapbox(df_score, lat="Latitude", lon="Longitude", hover_name="Name", color='score_inverted_normalized', color_continuous_scale=px.colors.cyclical.IceFire, zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()